This notebook is to generate hisoric data for us to analyse in a seperate notebook
### To do:
- Speed up the generator if possible

In [ ]:
#Import the libraries
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import math
from datetime import timedelta 
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
import cvxpy as cp
#from joblib import Parallel, delayed
#Import our homebrew functions
import DatabaseMainFnc as dmf

pd.options.mode.chained_assignment = None #silences setting with a copy warning
pd.set_option('display.max_columns', 38)
plt.rcParams['figure.figsize'] = [15, 10]


## Load, Validate & convert our price data to EUR

In [ ]:
%%time
#Load database
df_Nasdaq=dmf.connectAndLoadDb('NASDAQ')

#Set date as index
df_Nasdaq=df_Nasdaq.set_index(pd.DatetimeIndex(df_Nasdaq['Date'].values))
df_Nasdaq.drop(columns=['Date'],axis=1, inplace=True)

#Validate the data 
dmf.priceDB_validation(df_Nasdaq)

#Do currency conversion here, we want USD in EUR out
df_Nasdaq=dmf.load_curr_csv(df_Nasdaq,"USD")

#Show us the data
df_Nasdaq.tail(5)

## Demonstrate the generator fuction with info displayed

In [ ]:
%%time
database=df_Nasdaq
startdate=pd.to_datetime("2009-07-01")
enddate=pd.to_datetime("2019-01-01")
dmf.portfolio_generate_test(database,startdate,enddate,p_max=250, min_returns=0.01, s_asset=0, asset_len=150, silent=False, obj_method="SHARPE")

In [ ]:
%%time
#set or inputs & create empty lists for output
database=df_Nasdaq

#vary start date but beware its an N! problem need to speed this up if we can
startdate=pd.to_datetime("2006-01-01")
enddate=pd.to_datetime("2020-01-01")
a_test=[]
col_s_assets=[]
t_start=startdate
t_end=enddate

#vary start asset 0 takes the top 50 assets , 50 takes 50-99,.... this is an N problem which isnt too bad
l_s_asset=np.arange(0,550,50)


# example code that may also work for speeding this up rates_df[curr]=Parallel(n_jobs=-1)(delayed(get_rate)(curr,'EUR', date) for date in dates)
# runnings through starting assets, as we create portfolios for only 50 assets at a time (this can be varied)
for strt in l_s_asset: 
    t_start=startdate
    t_end=enddate
    print('Running for s_asset : '+ str(strt))

    # iterates through start dates by adding one month each time until theres 1 year between the start & end date
    while t_start + pd.DateOffset(years=1) <= t_end:
        try:
            a_test.append(dmf.portfolio_generate_test(database,t_start,t_end,s_asset=strt,asset_len=50))
            col_s_assets.append(strt)
            t_start=t_start + pd.DateOffset(months=1)

            #when our start date is 1 year from the end date subtract 1 year from the enddate & reset the start date
            if t_start + pd.DateOffset(years=1) == t_end:
                t_start=startdate
                t_end=t_end - pd.DateOffset(months=1)
        except:
            t_start=t_start + pd.DateOffset(months=1)

            #when our start date is 1 year from the end date subtract 1 year from the enddate & reset the start date
            if t_start + pd.DateOffset(years=1) == t_end:
                t_start=startdate
                t_end=t_end - pd.DateOffset(months=1)

#put all this data in a pandas datafame and save it             
df_test=pd.DataFrame(a_test,columns=['Start_Date','End_date','Expected_returns','volatility', 'sharpe_ratio','max_returns', 'min_returns', 'actual_returns','mean_returns','objective_summary'])
df_test['starting_asset']=col_s_assets
df_test.to_csv('Portfolio data/Nasdaq_EF_data.csv')

## Preview our data , more detailed analysis will be done in a seperate notebook

In [ ]:
print("Actual returns should be >= -1, we have : "+str(df_test['actual_returns'].min())+str(df_test['min_returns'].min()))
df_test.tail(3)

In [ ]:
# Plot expected vs actual scatter
import matplotlib.lines as mlines
ax1 = df_test.plot.scatter(x='Expected_returns', y='max_returns',alpha=0.5)

#set the x=y line
line = mlines.Line2D([0, 1], [0, 1], color='red')

#draw axis lines
plt.axhline(0,color='gray') # x = 0
plt.axvline(0,color='gray') # y = 0
transform = ax1.transAxes
line.set_transform(transform)
ax1.add_line(line)
ax1.set_xlim([-1, 6])
ax1.set_ylim([-1, 6])

## Do the same for NYSE

In [ ]:
%%time
#Load database
df_NYSE=dmf.connectAndLoadDb('NYSE')

#Set date as index
df_NYSE=df_NYSE.set_index(pd.DatetimeIndex(df_NYSE['Date'].values))
df_NYSE.drop(columns=['Date'],axis=1, inplace=True)

#Validate the data 
dmf.priceDB_validation(df_NYSE)

#Do currency conversion here, we want USD in EUR out
df_NYSE=dmf.load_curr_csv(df_NYSE,"USD")

#Show us the data
df_NYSE.tail(5)

In [ ]:
%%time
database=df_NYSE
startdate=pd.to_datetime("2009-07-01")
enddate=pd.to_datetime("2019-01-01")
dmf.portfolio_generate_test(database,startdate,enddate,p_max=250, min_returns=0.01, s_asset=0, asset_len=150, silent=False, obj_method="SHARPE")

In [ ]:
%%time
#set or inputs & create empty lists for output
database=df_NYSE

#vary start date but beware its an N! problem need to speed this up if we can
startdate=pd.to_datetime("2006-01-01")
enddate=pd.to_datetime("2020-01-01")
a_test=[]
col_s_assets=[]
t_start=startdate
t_end=enddate

#vary start asset 0 takes the top 50 assets , 50 takes 50-99,.... this is an N problem which isnt too bad
l_s_asset=np.arange(0,550,50)


# example code that may also work for speeding this up rates_df[curr]=Parallel(n_jobs=-1)(delayed(get_rate)(curr,'EUR', date) for date in dates)
# runnings through starting assets, as we create portfolios for only 50 assets at a time (this can be varied)
for strt in l_s_asset: 
    t_start=startdate
    t_end=enddate
    print('Running for s_asset : '+ str(strt))

    # iterates through start dates by adding one month each time until theres 1 year between the start & end date
    while t_start + pd.DateOffset(years=1) <= t_end:
        try:
            a_test.append(dmf.portfolio_generate_test(database,t_start,t_end,s_asset=strt,asset_len=50))
            col_s_assets.append(strt)
            t_start=t_start + pd.DateOffset(months=1)

            #when our start date is 1 year from the end date subtract 1 year from the enddate & reset the start date
            if t_start + pd.DateOffset(years=1) == t_end:
                t_start=startdate
                t_end=t_end - pd.DateOffset(months=1)
        except:
            t_start=t_start + pd.DateOffset(months=1)

            #when our start date is 1 year from the end date subtract 1 year from the enddate & reset the start date
            if t_start + pd.DateOffset(years=1) == t_end:
                t_start=startdate
                t_end=t_end - pd.DateOffset(months=1)

#put all this data in a pandas datafame and save it             
df_test=pd.DataFrame(a_test,columns=['Start_Date','End_date','Expected_returns','volatility', 'sharpe_ratio','max_returns', 'min_returns', 'actual_returns','mean_returns','objective_summary'])
df_test['starting_asset']=col_s_assets
df_test.to_csv('Portfolio data/NYSE_EF_data.csv')

In [ ]:
# Plot expected vs actual scatter
import matplotlib.lines as mlines
ax1 = df_test.plot.scatter(x='Expected_returns', y='max_returns',alpha=0.5)

#set the x=y line
line = mlines.Line2D([0, 1], [0, 1], color='red')

#draw axis lines
plt.axhline(0,color='gray') # x = 0
plt.axvline(0,color='gray') # y = 0
transform = ax1.transAxes
line.set_transform(transform)
ax1.add_line(line)
ax1.set_xlim([-1, 6])
ax1.set_ylim([-1, 6])

In [ ]:
df_test.tail()